In [ ]:
#-------Importing Packages---------#
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import cv2
import os
import bin2cell as b2c
import scipy

In [ ]:
#---------Setting up Directories--------#
#create directory for stardist input/output files
path = "/projects/b1217/HHA/EL_S2_B2C/"
os.chdir(path)
os.makedirs("stardist", exist_ok=True)

In [ ]:
#-------Building AnnData Object for 2 um Bins---------#
#Path to full resolution image
source_image_path = "/projects/b1217/Edward/Spatial/VisiumHD/Human/20240612EL_HuScalp_V2/outs/spatial/20241020_huscalp_EL_20x_tiled_HE_PS.tif"
#Creating Anndata
adata = b2c.read_visium("/projects/b1217/Edward/Spatial/VisiumHD/Human/20240612EL_HuScalp_V2/outs/binned_outputs/square_002um/", source_image_path = source_image_path)
adata.var_names_make_unique()
adata

In [ ]:
#------Filtering for Spots/Gene Expression--------#
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_counts=1)
adata

In [ ]:
#--------Saving Scaled Image----------#
#set microns per pixel measurement
mpp = 0.5
b2c.scaled_he_image(adata, mpp=mpp, save_path="stardist/he.tiff")

In [ ]:
#----------Destriping Bins-------#
#This will help with GEX-based segmentation
#Currently not adjusting counts to preserve integer matrix for SCVI,TACCO
b2c.destripe(adata, adjust_counts=False)
adata.obs.head()

In [ ]:
#--------Defining Crop-----------#
#define a mask to easily pull out this region of the object in the future
#currently finding the coordinates by trial and error; will find way to display coordinates for better subsetting
mask = ((adata.obs['array_row'] >= 2800) & 
        (adata.obs['array_row'] <= 3200) & 
        (adata.obs['array_col'] >= 2520) & 
        (adata.obs['array_col'] <= 3020))

#---------Plotting Crop---------#
bdata = adata[mask]
sc.set_figure_params(figsize=[10,10],dpi=100)
sc.pl.spatial(bdata, color=[None, "n_counts", "n_counts_adjusted"], img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer", save='_spatial_destriping.pdf', cmap = "Reds")

In [ ]:
#---------Running H&E-based Segmentation--------#
#run stardist segmentation tool
b2c.stardist(image_path="stardist/he.tiff", 
             labels_npz_path="stardist/he.npz", 
             stardist_model="2D_versatile_he", 
             prob_thresh=0.01)

In [ ]:
#---------Labeling bins by Segmentation--------#
#insert stardist results into spatial anndata object
b2c.insert_labels(adata, 
                  labels_npz_path="stardist/he.npz", 
                  basis="spatial", 
                  spatial_key="spatial_cropped_150_buffer",
                  mpp=mpp, 
                  labels_key="labels_he")

In [ ]:
#---------Plotting Segmented Cells--------#
#the label viewer wants a crop of the processed image
#Cropping to masked region
seg_crop = adata[mask]
seg_crop = seg_crop[seg_crop.obs['labels_he']>0]
seg_crop.obs['labels_he'] = seg_crop.obs['labels_he'].astype(str)
#get the corresponding coordinates spanning the subset object
crop = b2c.get_crop(seg_crop, basis="spatial", spatial_key="spatial_cropped_150_buffer", mpp=mpp)
rendered = b2c.view_stardist_labels(image_path="stardist/he.tiff", 
                                    labels_npz_path="stardist/he.npz", 
                                    crop=crop)
plt.imshow(rendered)
plt.savefig('he_labels_image.pdf')

In [ ]:
#---------Plotting Segmented Cells--------#
#the label viewer wants a crop of the processed image
#Cropping to masked region
seg_crop = adata[mask]
seg_crop = seg_crop[seg_crop.obs['labels_he']>0]
seg_crop.obs['labels_he'] = seg_crop.obs['labels_he'].astype(str)
#get the corresponding coordinates spanning the subset object
crop = b2c.get_crop(seg_crop, basis="spatial", spatial_key="spatial_cropped_150_buffer", mpp=mpp)
rendered = b2c.view_stardist_labels(image_path="stardist/he.tiff", 
                                    labels_npz_path="stardist/he.npz", 
                                    crop=crop)
plt.imshow(rendered)
plt.savefig('he_labels_image.pdf')

In [ ]:
#---------Expanding Segmentations to Surrounding Bins--------#
#Using default method of adding the neighboring two bins, using PCA distance to resolve ownership disputes
b2c.expand_labels(adata, 
                  labels_key='labels_he', 
                  expanded_labels_key="labels_he_expanded")

In [ ]:
adata

In [ ]:
#---------Plotting Expanded Segmentations--------#
#the label viewer wants a crop of the processed image
bdata = adata[mask]
bdata = bdata[bdata.obs['labels_he']>0]
bdata.obs['labels_he'] = bdata.obs['labels_he_expanded'].astype(str)
#get the corresponding coordinates spanning the subset object
crop = b2c.get_crop(bdata, basis="spatial", spatial_key="spatial_cropped_150_buffer", mpp=mpp)
sc.pl.spatial(bdata, color=[None, "labels_he_expanded"], img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer")

In [ ]:
#-----------Aggregating Binned Data into Cells----------#
cdata = b2c.bin_to_cell(adata, labels_key="labels_he_expanded", spatial_keys=["spatial", "spatial_cropped_150_buffer"])
cdata

In [ ]:
#---------Plotting Cell-Level Data----------#
#show masking based on cell
cell_mask = ((cdata.obs['array_row'] >= 2800) & 
             (cdata.obs['array_row'] <= 3200) & 
             (cdata.obs['array_col'] >= 2520) & 
             (cdata.obs['array_col'] <= 3020))
ddata = cdata[cell_mask]
sc.set_figure_params(fontsize=20,figsize=[7,7])
sc.pl.spatial(ddata, color=["bin_count"], img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer", s=10)

In [ ]:
#----------Saving Cell and Spot-Level Data-------#
#Spots
adata.write_h5ad('/projects/b1217/HHA/EL_S2_B2C/anndatas/B2C_HE_Segmentation_NoDestripe_Spot_adata_11_13_25.h5ad')
#Cells
cdata.write_h5ad('/projects/b1217/HHA/EL_S2_B2C/anndatas/B2C_HE_Segmentation_NoDestripe_Cell_adata_11_13_25.h5ad')